<a href="https://colab.research.google.com/github/hryndel2/AI_ISMA/blob/main/photo_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout, Dense, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Define data parameters
batch_size = 32
img_height = 180
img_width = 180

# Define data directory
data_dir = 'data/image'

# Define data augmentation
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  tf.keras.layers.experimental.preprocessing.RandomTranslation(0.1, 0.1)
])

# Define image generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4
)

# Define train and validation datasets
train_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Define text tokenizer
tokenizer = Tokenizer()
texts = ['_'.join(train_ds.filenames)]
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=100)

# Define neural network model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='leaky_relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='leaky_relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), activation='leaky_relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='leaky_relu'))
model.add(Dropout(0.5))

# Get the number of classes from the class indices
num_classes = len(train_ds.class_indices)

# Define the output layer with softmax activation
model.add(Dense(num_classes, activation='softmax'))

# Compile neural network model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train neural network model
history = model.fit(
    train_ds,
    epochs=100,
    validation_data=val_ds
)

# Save the trained model
model.save('paint_bot.h5')

# Define a function to generate a new image based on user input
def generate_image():
    # Get user input for the desired text
    desired_text = input('Enter the desired text: ')

    # Generate a new image
    new_image = np.random.rand(1, img_height, img_width, 3) * 255

    # Apply data augmentation to the new image
    augmented_image = data_augmentation(new_image)

    # Generate a prediction for the new image
    prediction = model.predict(augmented_image)

    # Get the predicted class
    predicted_class = np.argmax(prediction)

    # Check if the predicted class matches the desired text
    if tokenizer.index_word[predicted_class] == desired_text:
        # Save the new image to the "result" folder
        if not os.path.exists('result'):
            os.makedirs('result')

        filename = 'result/generated_image_' + str(predicted_class) + '.png'

        # Save the augmented image to the "result" folder
        tf.keras.preprocessing.image.save_img(filename, augmented_image[0])

        print('Generated image saved to: ' + filename)
    else:
        print('The generated image does not match the desired text. Please try again.')

# Call the generate_image function to start the image generation process
while True:
    generate_image()

FileNotFoundError: [Errno 2] No such file or directory: 'data/images'